<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/167_Conditional_Routing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This `customer_service.py` example is a **teaching demo** for **conditional routing** in LangGraph. Let’s break down what you should be focusing on and learning here:

---

## 🔑 1. State Definition

```python
class SupportRequest(TypedDict):
    message: str
    priority: int
```

* This defines the **state schema** (the data flowing through the graph).
* Every node will receive this `SupportRequest` and can return updates to it.

👉 **Lesson:** State is strongly typed and explicit, so you know what information nodes can rely on.

---

## 🔑 2. Conditional Router Function

```python
def categorize_request(request: SupportRequest):
    if "urgent" in request['message'].lower() or request['priority'] == 1:
        return "high"
    return "low"
```

* This function doesn’t modify state; instead, it **returns a label** (`"high"` or `"low"`).
* That label is matched against the mapping in `add_conditional_edges`.

👉 **Lesson:** Routing decisions can be **arbitrary Python logic**. The router is where you encode **business rules**.

---

## 🔑 3. Nodes for Each Path

```python
def handle_urgent(request): ...
def handle_standard(request): ...
```

* Each path (urgent vs standard) has its **own node function**.
* They both receive the same state but are executed based on routing.

👉 **Lesson:** You don’t build one giant “if/else” function. Instead, you split logic into **modular nodes**, making the graph cleaner and easier to extend.

---

## 🔑 4. Graph Wiring

```python
graph.add_node("urgent", handle_urgent)
graph.add_node("standard", handle_standard)

graph.add_conditional_edges(
    START, categorize_request,
    {"high": "urgent", "low": "standard"}
)
graph.add_edge("urgent", END)
graph.add_edge("standard", END)
```

* `categorize_request` decides the next node.
* The dictionary maps decision values (`"high"`, `"low"`) to node names.
* Each node routes to `END` after finishing.

👉 **Lesson:** `add_edge` = fixed, linear flow.
`add_conditional_edges` = **dynamic branching** (execution depends on runtime state).

---

## 🔑 5. Running the Graph

```python
print(runnable.invoke({"message": "My account was hacked! Urgent help needed.", "priority": 1}))
print(runnable.invoke({"message": "I need help with password reset.", "priority": 3}))
```

* First request → `"urgent"` path.
* Second request → `"standard"` path.

👉 **Lesson:** The same graph handles **different cases automatically**, without separate pipelines.

---

## 🧠 What to Take Away

1. **Conditional routing = dynamic branching**: one entry point, multiple possible paths.
2. **Separation of concerns**: router decides, specialized nodes act.
3. **Declarative orchestration**: graph edges describe the workflow instead of hardcoding `if/else` in one place.
4. **Extensibility**: easy to add new categories (`medium`, `VIP`, etc.) by extending the router + adding nodes.

---

✅ This is the foundation for **more advanced routing agents** (like triaging medical cases, routing queries to expert agents, or picking tools in an MoE setup).




In [ ]:
from typing import TypedDict
from langgraph.graph import END, START, StateGraph


# Define the structure of the input state (customer support request)
class SupportRequest(TypedDict):
    message: str
    priority: int  # 1 (high), 2 (medium), 3 (low)


# Function to categorize the support request
def categorize_request(request: SupportRequest):
    print(f"Received request: {request}")
    # TODO: Implement Conditional Routing
    if "urgent" in request['message'].lower() or request['priority'] == 1:
        return "high"
    return "low"


# Function to process high-priority requests
def handle_urgent(request: SupportRequest):
    print(f"Routing to Urgent Support Team: {request}")
    return request


# Function to process standard requests
def handle_standard(request: SupportRequest):
    print(f"Routing to Standard Support Queue: {request}")
    return request


# Create the state graph
graph = StateGraph(SupportRequest)
# TODO: Create the graph
graph.add_node("urgent",handle_urgent)
graph.add_node("standard",handle_standard)

graph.add_conditional_edges(START,categorize_request,{"high":"urgent","low":"standard"})
graph.add_edge("urgent",END)
graph.add_edge("standard",END)

runnable = graph.compile()

# Simulate a customer support request
print(runnable.invoke({"message": "My account was hacked! Urgent help needed.", "priority": 1}))
print(runnable.invoke({"message": "I need help with password reset.", "priority": 3}))


## 🔹 In plain Python (without LangGraph)

Normally, if you wrote a support workflow, you might do:

```python
def handle_request(request):
    if "urgent" in request["message"].lower() or request["priority"] == 1:
        # urgent branch
        do_high_priority_action(request)
    else:
        # standard branch
        do_low_priority_action(request)
```

This means:

* All the logic lives inside **one big function**.
* Adding more categories later (`VIP`, `medium`, `follow_up`, etc.) makes the function more and more cluttered.
* Harder to test, harder to extend, less modular.

---

## 🔹 In LangGraph

Instead, you model the workflow as **nodes**:

```python
graph.add_node("urgent", handle_urgent)
graph.add_node("standard", handle_standard)
```

Each node is just a **small, focused function** that:

* Accepts the same state (e.g., `SupportRequest` with `message` and `priority`).
* Does its own specific job (urgent handling vs standard handling).
* Returns an updated state.

Then, you wire them together:

```python
graph.add_conditional_edges(
    START,
    categorize_request,
    {"high": "urgent", "low": "standard"}
)
```

Now, only the relevant branch runs:

* If the router says `"high"` → only the `"urgent"` node executes.
* If the router says `"low"` → only the `"standard"` node executes.
* The other path is **never touched**.

---

## 🔹 Why this is better

1. **Modularity**
   Each node is independent. You can test `handle_urgent` in isolation without worrying about the rest.

2. **Clarity**
   The workflow diagram shows exactly which parts exist and how they’re connected.
   No digging through nested `if/else` statements.

3. **Extensibility**
   If you later add `"medium"`, you just:

   * Create `handle_medium` node.
   * Add it to the graph.
   * Update the router mapping.
     No need to rewrite old logic.

4. **Dynamic execution**
   When the graph runs, only the path determined by the router is executed. This makes it efficient — you’re not evaluating unnecessary branches.

---

### 🧠 The key idea:

Think of the graph as **designing the full map of possible workflows**.
But when you actually run it, **only one route through the map is taken**, depending on the data (the state).




The **modular node-based approach** gives you three big advantages over writing everything as one giant `if/else` function:

---

### 1. **Testability in Isolation**

Each node is just a plain Python function.
You can unit test `handle_urgent(request)` separately from `handle_standard(request)` without having to simulate the entire workflow.

That means if something breaks, you know *exactly* which piece of the puzzle caused it.

---

### 2. **Extensibility Without Breaking Things**

Say today you only need `"urgent"` vs `"standard"`. Tomorrow, your manager says:

> “Add a `VIP` tier that jumps straight to a manager.”

In the monolithic approach, you’d be rewriting your big `if/else` block and risk accidentally breaking the `"urgent"` or `"standard"` code.

In the graph approach, you just:

* Add a new node `handle_vip`,
* Add `"vip": "handle_vip"` to the router mapping,
* Done ✅.

All your old logic remains untouched.

---

### 3. **Cleaner & Easier to Understand**

Instead of reading a 100-line `if/else` jungle, you look at a **graph of nodes**:

```
START
   ↓
categorize_request
   ↙          ↘
urgent      standard
   ↓           ↓
 END          END
```

Anyone can glance at that and know what the workflow does.
It also scales: 10 branches are still easy to visualize in a graph, but a nightmare in nested Python conditionals.

---

### 🧠 Rule of Thumb

* Use **monolithic `if/else`** for tiny scripts (1–2 branches).
* Use **modular graph routing** once you want **reusability, clarity, or future extensions**.




## 📝 Suggested structure for your LangGraph Agent Guide (`AGENT_GUIDE.md`)

Here’s a clean outline we can start with:

### 1. **Introduction**

* Purpose of this guide (standardize how we build LangGraph agents).
* Core principles: modularity, reusability, clarity, testability.

---

### 2. **State Design**

* Always define a `TypedDict` or extend `MessagesState`.
* Use reducers (`Annotated[..., add]`) for accumulating fields.
* Include examples:

  ```python
  from typing import TypedDict, Annotated
  from operator import add
  from langchain_core.messages import AnyMessage

  class AgentState(TypedDict):
      messages: Annotated[list[AnyMessage], add]
      task_id: Annotated[int, add]
      status: str
  ```

---

### 3. **Node Design**

* One node = one clear responsibility.
* Nodes are **small Python functions**: input = state, output = updated state.
* Prefer small, testable functions over giant ones.

Example:

```python
def verify_insurance(state: AgentState) -> AgentState:
    # Logic here
    state["status"] = "APPROVED"
    return state
```

---

### 4. **Graph Wiring**

* Use `add_edge` for linear flows.
* Use `add_conditional_edges` for branching logic.
* Show a sample:

  ```python
  builder.add_edge("verify", "schedule")

  builder.add_conditional_edges(
      "categorize",
      lambda s: s["priority"],
      {"urgent": "urgent_node", "standard": "standard_node"}
  )
  ```

---

### 5. **Subgraphs**

* Use subgraphs for reusable components (insurance check, compliance validation).
* Rule of thumb: if 2+ workflows repeat the same steps → make a subgraph.

---

### 6. **Tools & RAG Integration**

* Use `ToolNode` for dynamic tool execution (vs static nodes).
* Consider RAG nodes when working with evolving documents (compliance, FAQs).

---

### 7. **Error Handling**

* Built-in: `retry_node` wrapper for resilience.
* Encourage structured exception handling inside nodes where external APIs are called.

---

### 8. **Testing & Debugging**

* Each node should be testable in isolation.
* Use small unit tests for nodes.
* Test whole workflow with sample states.

---

### 9. **Human-in-the-Loop**

* Insert `interrupt` points where human validation is needed.
* Example use cases: compliance approvals, critical decisions.

---

### 10. **Best Practices**

* Keep nodes modular and single-purpose.
* Prefer clarity over cleverness.
* Use ASCII diagrams or `graph.get_graph().draw()` for visualization.
* Document assumptions in comments.

---

⚡️ With this structure, your `.MD` file will become a **living standard** for building agents. You can even extend it with sections like *naming conventions* or *file organization* if you want Claude to stick to a particular repo style.


# LangGraph Agent Design Guide

This guide provides design specifications and best practices for building LangGraph agents. Use this as a reference when building agents in Cursor or collaborating with Claude Code.

---

## 1. Introduction

LangGraph enables modular, stateful, and composable AI workflows. This guide ensures agents are built consistently, with a focus on:

* **Modularity** → Each node has a single responsibility.
* **Reusability** → Common patterns abstracted into subgraphs.
* **Clarity** → Workflows are easy to understand and extend.
* **Testability** → Each node and workflow can be tested in isolation.

---

## 2. State Design

Every agent defines a state schema. Use a `TypedDict` or extend `MessagesState`.

### Example with custom TypedDict

```python
from typing import TypedDict, Annotated
from operator import add
from langchain_core.messages import AnyMessage

class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], add]
    task_id: Annotated[int, add]
    status: str
```

### Example extending MessagesState

```python
from langgraph.graph import MessagesState

class OrderState(MessagesState):
    order_id: Annotated[int, add]
```

**Rule of Thumb:**

* Use `MessagesState` if conversation history is central.
* Use custom `TypedDict` if state is structured data (IDs, statuses, items).

---

## 3. Node Design

Nodes are small, focused Python functions. Input = state, Output = updated state.

```python
def verify_insurance(state: AgentState) -> AgentState:
    # Business logic here
    state["status"] = "APPROVED"
    return state
```

✅ Each node should be independently testable.

---

## 4. Graph Wiring

Use **linear edges** for fixed flows and **conditional edges** for branching.

```python
# Linear
builder.add_edge("verify", "schedule")

# Conditional
builder.add_conditional_edges(
    "categorize",
    lambda s: s["priority"],
    {"urgent": "urgent_node", "standard": "standard_node"}
)
```

Think of the graph as a **map of possible paths**. Execution follows only one path at runtime, depending on state.

---

## 5. Subgraphs

Subgraphs are reusable components that appear as a single node inside a parent graph.

**Use Cases:**

* Insurance verification (healthcare)
* Compliance validation
* Authentication steps

Rule: If 2+ workflows reuse the same logic → make a subgraph.

---

## 6. Tools & RAG Integration

* **ToolNode** → Enables the model to dynamically call tools.

```python
from langgraph.prebuilt import ToolNode

tools = [check_symptoms, book_appointment]
tool_node = ToolNode(tools)
```

* **RAG Nodes** → Query vector stores or document indexes when knowledge must be current (e.g., compliance rules, FAQs).

---

## 7. Error Handling

LangGraph offers built-in resilience:

* **`retry_node`** → Automatically retries a node on failure.
* Manual `try/except` inside nodes for API calls.

```python
from langgraph.utils import retry_node

@retry_node(retries=3)
def call_api(state):
    # external API call
    ...
```

---

## 8. Testing & Debugging

* **Node Testing** → Call node functions with mock state.
* **Workflow Testing** → Run compiled graph with example input.
* **Visualization** → Use `.get_graph().draw()` for ASCII or graphviz diagrams.

```python
workflow.get_graph().print_ascii()
```

---

## 9. Human-in-the-Loop

Insert **interrupts** for human validation:

```python
from langgraph.checkpoint import interrupt

def approve_output(state):
    decision = interrupt("Approve generated output?")
    state["approved"] = decision
    return state
```

Use for:

* Compliance approvals
* Critical decision reviews

---

## 10. Best Practices

* Keep nodes **single-purpose**.
* Prefer **clarity over cleverness**.
* Always document assumptions.
* Modularize common steps into subgraphs.
* Log retrieved documents when using RAG for auditability.

---

## Example Workflow Diagram

```
START
  ↓
categorize_request
  ↙          ↘
urgent      standard
  ↓           ↓
handle_urgent  handle_standard
  ↓           ↓
   END       END
```

---

✅ With this guide, all LangGraph agents can be built in a consistent, modular, and maintainable way.
